In [227]:
from PIL import Image
import numpy as np
from Crypto.Hash import SHA512
import warnings
from itertools import chain

In [87]:
im = Image.open('./dendrites/2.tif')
im2arr = np.array(im) # im2arr.shape: height x width x channel
arr2im = Image.fromarray(im2arr)
read_len_pixel = 25

In [45]:
ip = int(input('Choose "1" for random selection, "2" for determistic selection, "0" for documents about the choice\n'))

Choose "1" for random selection, "2" for determistic selection, "0" for documents about the choice
 1


In [39]:
ip_set = [x for x in range(0,3)]

In [46]:
while ip not in ip_set:
    warnings.warn("Unknown command, plz retype the selection mode\n")
    ip = input('Choose "1" for random selection, "2" for determistic selection, "0" for documents about the choice\n')

In [111]:
def rand_sel(read_len_pixel=10, loc=None):
    print('-'*20)
    print('**Random selecting**')
    h,w,c = im2arr.shape
    if loc == None:
        temp_a = int(np.random.randint(h, size=1))    
        temp_b = int(np.random.randint(w, size=1))
        dir_sel = int(np.random.randint(2,size=1))
    else:
        temp_a, temp_b, dir_sel = loc

    if dir_sel == 0:
        if temp_b < w-read_len_pixel:
            sel_vec = np.mean(im2arr[temp_a, temp_b:temp_b+read_len_pixel,:], axis=1)
        else:
            sub = read_len_pixel - w + temp_b 
            sel_vec = np.mean(im2arr[temp_a, temp_b:,:], axis=1)
            if temp_a < h - 1:
                sel_vec = np.concatenate([sel_vec, np.mean(im2arr[temp_a+1, :sub,:], axis=1)])
            else:
                sel_vec = np.concatenate([sel_vec, np.mean(im2arr[0, :sub,:], axis=1)])
    else:
        if temp_a < h - read_len_pixel:
            sel_vec = np.mean(im2arr[temp_a:temp_a+10, temp_b,:], axis=1)
            # debug
        else:
            sub = read_len_pixel - h + temp_a
            sel_vec = np.mean(im2arr[temp_a:, temp_b,:], axis=1)
            if temp_b < w - 1:
                sel_vec = np.concatenate([sel_vec, np.mean(im2arr[:sub, temp_b+1,:], axis=1)])
            else:
                sel_vec = np.concatenate([sel_vec, np.mean(im2arr[:sub, 0,:], axis=1)])
    sel_vec = sel_vec.round().astype(int)
    return sel_vec

In [228]:
def hash(sel_vec):
    sel_str = [""+str(x) for x in sel_vec]
    hash_object = SHA512.new(data=(''.join(sel_str)).encode())
    return(hash_object)

In [229]:
hash_object = hash(sel_vec)
out = hash_object.hexdigest()
out

'7b0755f5b96b9910b268ac607dddeebcca72b238a9184cb1de3f768ed9989fda2e6b28512a717c60c51b8316c37f051ac3a9f9c34821ae6d4424f137369f2eb4'

In [233]:
def conversion(out_mode, hash_object, length_required):
    """
    out_mode : 0 -- binary
             : 1 -- decimal
             : 2 -- hex
             : 3 -- character
    return   : output in string format
    """
    hex_out = hash_object.hexdigest()
    bin_out = bin(int(hex_out, 16))[2:].zfill(len(hex_out)*4)
    if out_mode == 0:
        return(bin_out[-length_required:])
    elif out_mode == 2:
        return(hex_out[-length_required:])
    elif out_mode == 1:
        return(str(int(bin_out, 2))[-length_required:])
    elif out_mode == 3:
        hex_vec = []
        for i in range(int(len(hex_out) / 2)): 
            idx = 2 * i
            hex_vec.append(int(hex_out[idx:idx+2],16))
        chr_vec = [chr(x) for x in hex_vec]
        d = ''.join(chr_vec)
        concatenated = chain(range(48,58), range(65, 91), range(97, 123))
        availset = [chr(x) for x in concatenated]
        chr_vec_sel = ''.join([x for x in d if x in availset])
        while (length_required > len(chr_vec_sel)):
            length_required = int(input('Input a length which needs to be smaller than {}'.format(len(chr_vec_sel)+1)))
        return(chr_vec_sel[-length_required:])

In [234]:
conversion(3, hash_object, 32)

Input a length which needs to be smaller than 16 15


'Ukhr8LvkQqHmD76'